In [1]:
from pathlib import Path
import os
import sys
sys.path.append(str(Path(os.path.abspath('')).parent))
print(str(Path(os.path.abspath('')).parent))

/home/amai/splinetracers


In [63]:

from splinetracers.ellipsoid_splinetracer import trace_rays
import torch
import numpy as np
from icecream import ic


SH_C0 = 0.28209479177387814
N = 3
device = torch.device("cuda")

def l2_normalize_th(x, eps=torch.finfo(torch.float32).eps):
    """Normalize x to unit length along last axis."""
    return x / torch.sqrt(
        torch.clip(torch.sum(x**2, dim=-1, keepdim=True), eps, None)
    )

@torch.jit.script
def inv_opacity(y):
    x = (-(1 - y).clip(min=1e-10).log()).clip(min=0)
    return x

def inverse_sigmoid(x):
    return torch.log(x/(1-x))

scene_scale = 1.00
# opacities = Parameter(-torch.tensor(-0.0 + 1*np.random.rand(N), dtype=torch.float32, device=device))
# opacities = 0.005+torch.tensor(0.0*np.ones((N)), dtype=torch.float32, device=device)
# opacities = torch.tensor([0.1, 10, 0.1], dtype=torch.float32, device=device)
opacities = inverse_sigmoid(torch.tensor([0.1, 0.9, 0.1], dtype=torch.float32, device=device))
ic(opacities)
# opacities = Parameter(np.exp(3.25)+torch.tensor(0.2*np.random.rand(N), dtype=torch.float32, device=device))
# opacities = Parameter(-0.05+torch.tensor(0.2*np.random.rand(N), dtype=torch.float32, device=device))
# means = torch.tensor(np.array([[0.0, 0.0, 6], [0.0, 0.0, 5.1]]), dtype=torch.float32, device=device)
means = torch.tensor(np.array(
    [[0.0, 0.0, 40.0],
     [0.0, 0.0, 25.0],
     [0.0, 0.0, 0.2]]), dtype=torch.float32, device=device)
# means = torch.tensor(np.array([[0.0, 0.89, 6]]), dtype=torch.float32, device=device)
# scales = (-1.5-1*torch.tensor(np.random.rand(N, 3), dtype=torch.float32, device=device)).exp()
scales = inverse_sigmoid(0.90*torch.ones((N, 3), dtype=torch.float32, device=device))
quats = l2_normalize_th(2*torch.tensor(np.random.rand(N, 4), dtype=torch.float32, device=device)-1)
# quats = torch.tensor([[0.0,0,0,1]], device=device).expand(N, -1)
# means = torch.nextafter(means, torch.inf*torch.ones_like(means))
# quats[:, :3] = 0
# quats[:, 3] = 1
# quats = Parameter(l2_normalize_th(torch.tensor([[0, 0, 0, 1]], dtype=torch.float32, device=device)))
feats = torch.zeros((N, 1, 3), dtype=torch.float32, device=device)
# feats[:, 0:1, :] = torch.tensor(np.ones((N, 1, 3)), dtype=torch.float32, device=device)
a = torch.tensor(np.eye(3), dtype=torch.float32, device=device)
feats[:, 0, :] = (a - 0.5) / SH_C0
# feats[:, 0, :] = a

means = torch.nn.Parameter(means[:, 2])
scales = torch.nn.Parameter(scales)
quats = torch.nn.Parameter(quats)
feats = torch.nn.Parameter(feats)
opacities = torch.nn.Parameter(opacities)

input_densities = inv_opacity(torch.sigmoid(opacities)) / 2 if trace_rays.uses_density else torch.sigmoid(opacities)
# input_densities.retain_grad()
rayo = torch.tensor([
    [0.0, 0.0, 0],
], dtype=torch.float32, device=device)
rayd = torch.tensor([
    [0.0, 0, 1],
], dtype=torch.float32, device=device)

ic| opacities: tensor([-2.1972,  2.1972, -2.1972], device='cuda:0')


In [66]:

optim = torch.optim.Adam([
    means, scales, opacities
], lr=1e-2)

ic(means, scales, torch.sigmoid(opacities))
for i in range(100):
    input_densities = inv_opacity(torch.sigmoid(opacities)) / 2 if trace_rays.uses_density else torch.sigmoid(opacities)
    full_means = torch.zeros_like(scales)
    full_means[:, 2] = means
    color = trace_rays(
        full_means, torch.sigmoid(scales), quats,
        input_densities,
        feats,
        rayo,
        rayd)

    loss = color[..., 4].sum()
    loss.backward()
    optim.step()
    optim.zero_grad(set_to_none=True)
print(loss)

ic| means: Parameter containing:
           tensor([39.1590, 24.4411, -0.7128], device='cuda:0', requires_grad=True)
    scales: Parameter containing:
            tensor([[1.2418, 1.2418, 1.2419],
                    [2.5583, 2.5583, 2.5583],
                    [1.1593, 1.1593, 1.1593]], device='cuda:0', requires_grad

=True)
    torch.sigmoid(opacities): tensor([0.0461, 0.9294, 0.0542], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor(0.0149, device='cuda:0', grad_fn=<SumBackward0>)


In [67]:
ic(means, torch.sigmoid(scales), torch.sigmoid(opacities))
ic(color)
ic(feats)
input_densities = inv_opacity(torch.sigmoid(opacities)) / 2 if trace_rays.uses_density else torch.sigmoid(opacities)
color, extras = trace_rays(
    full_means, torch.sigmoid(scales), quats,
    input_densities,
    feats,
    rayo,
    rayd, return_extras=True)
print(color)

ic| means: Parameter containing:
           tensor([38.4445, 25.1827, -0.8473], device='cuda:0', requires_grad=True)
    torch.sigmoid(scales): tensor([[0.6107, 0.6107, 0.6108],
                                   [0.9614, 0.9614, 0.9614],
                                   [0.7359, 0.7359, 0.7359]], device='cuda:0', grad_fn=<SigmoidBackward0>)
    torch.sigmoid(opacities): tensor([0.0234, 0.9645, 0.0489], device='cuda:0', grad_fn=<SigmoidBackward0>)
ic| color: tensor([[ 5.8622e-04,  9.5941e-01, -7.0793e-09,  9.6000e-01,  1.4928e-02]],
                  device='cuda:0', grad_fn=<SplineTracerBackward>)
ic| feats: Parameter containing:
           tensor([[[ 1.7725, -1.7725, -1.7725]],
           
                   [

[-1.7725,  1.7725, -1.7725]],
           
                   [[-1.7725, -1.7725,  1.7725]]], device='cuda:0', requires_grad=True)


tensor([[ 5.7975e-04,  9.5960e-01, -7.0807e-09,  9.6018e-01,  1.4766e-02]],
       device='cuda:0', grad_fn=<SplineTracerBackward>)
